In [110]:
M = [
    #0  #1 #2 #3
    [0, 1, 0, 1], # 0
    [1, 0, 1, 0], # 1
    [0, 1, 0, 0], # 2
    [1, 0, 0, 0], # 3
]
"""
0 — 1
|   |
3   2
"""

class Node(object):
    def __init__(self, val = 0, neighbors = None):
        self.val = val
        self.neighbors = neighbors if neighbors is not None else []

def matrix2Graph(M): # directed or undirected
    nodeList = [ Node(i) for i in range(len(M)) ]
    for i in range(len(M)):
        for j in range(len(M[0])):
            if M[i][j]:
                nodeList[i].neighbors.append( nodeList[j] )
    return nodeList[0]
    
G = matrix2Graph(M)
print(G.val, G.neighbors, [node.val for node in G.neighbors] )

0 [<__main__.Node object at 0x7fa21896d250>, <__main__.Node object at 0x7fa21896da00>] [1, 3]


In [131]:
G = matrix2Graph(M)
def dfs(G):
    ans, stack, visit = [], [G], {id(G)}
    def search():
        nonlocal ans, stack, visit
        if stack:
            node = stack.pop()
            ans.append( node.val )
            for neighbor in filter(lambda N:id(N) not in visit, reversed(node.neighbors)):
                visit.add( id(neighbor) )
                stack.append( neighbor )
            search()
    search()
    return ans

print( dfs(G) )

[0, 1, 2, 3]


In [132]:
def bfs(G):
    ans, queue, visit = [], deque([G]), {id(G)}
    def search():
        nonlocal ans, queue, visit
        if queue:
            node = queue.popleft()
            ans.append( node.val )
            for neighbor in filter(lambda N:id(N) not in visit, node.neighbors):
                visit.add( id(neighbor) )
                queue.append( neighbor )
            search()     
    search()
    return ans

print( bfs(G) )

[0, 1, 3, 2]


In [134]:
"""
G:
0 — 1
|   |
3   2
G.neighbors = [ node1, node3 ]
G2: 4 - 1
G3: 5
"""
G = matrix2Graph(M)
G2 = Node(4, [G.neighbors[0]] )
G3 = Node(5, [])
G.neighbors[0].neighbors.append(G2)
print(dfs(G), bfs(G))

def isConnect(G1, G2):
    if id(G1)==id(G2):
        return True
    stack, visit, connect = [G1], {id(G1)}, False
    def dfs():
        nonlocal stack, visit, connect
        if not connect and stack:
            node = stack.pop()
            for neighbor in filter(lambda N:id(N) not in visit, node.neighbors):
                if id(neighbor)==id(G2):
                    connect = True 
                    return
                else:
                    visit.add(id(neighbor))
                    stack.append(neighbor)
            dfs()
    dfs()
    return connect
print( isConnect(G,G2), isConnect(G,G3) )

[0, 1, 2, 4, 3] [0, 1, 3, 2, 4]
True False


In [138]:
"""
G       E
0 — 1   0 — 1
|   |   |   |
3   2   3 — 2
"""
G = matrix2Graph(M)
E = matrix2Graph(M)
E.neighbors[0].neighbors[1].neighbors.append( E.neighbors[1] )  # 2->3
E.neighbors[1].neighbors.append( E.neighbors[0].neighbors[1] ) # 3->2
print( dfs(E), bfs(E) )

def hasCycle(G):
    stack, visit, cycle = [G], {id(G)}, False
    def dfs(lastID=None):
        nonlocal stack, visit, cycle
        if not cycle and stack:
            node = stack.pop()
            for neighbor in node.neighbors: 
                if id(neighbor)==lastID:    # exclude trace parent node
                    continue                
                elif id(neighbor) in visit: # don't use visit filter because cycle should be set as True
                    cycle = True
                    return
                else:
                    visit.add( id(neighbor) ) 
                    stack.append( neighbor )
                    dfs( lastID=id(node) )  # dfs must implement directly due to unovewritable lastID
    dfs()
    return cycle
print(hasCycle(G), hasCycle(E))

[0, 1, 2, 3] [0, 1, 3, 2]
False True


In [139]:
G = matrix2Graph(M)
def graph2Matrix(G,n):
    M = [ [0]*n for i in range(n) ]
    stack, visit = [G], {id(G)}
    def dfs():
        nonlocal M, stack, visit
        if stack:
            node = stack.pop()
            for neighbor in filter(lambda N:id(N) not in visit, node.neighbors):
                M[node.val][neighbor.val], M[neighbor.val][node.val] = 1, 1
                visit.add( id(neighbor) )
                stack.append( neighbor )
            dfs()
    dfs()
    return M

for row in graph2Matrix(G,4):
    print(row)

[0, 1, 0, 1]
[1, 0, 1, 0]
[0, 1, 0, 0]
[1, 0, 0, 0]


In [142]:
"""
G       spanningTree
0 — 1   0 — 1
| \ |   |   |
3   2   3   2
# v nodes -> spanning tree has v-1 edges
"""
G = matrix2Graph(M)
G.neighbors.append( G.neighbors[0].neighbors[1] ) # 0->2
G.neighbors[0].neighbors[1].neighbors.append( G ) # 2->0

def spanningTree(G):
    T = Node(G.val)
    stack, visit = [G], {id(G)}
    def dfs(p=T):
        nonlocal stack, visit
        if stack:
            node = stack.pop()
            for neighbor in filter(lambda N:id(N) not in visit, node.neighbors):
                visit.add( id(neighbor) )
                stack.append( neighbor )
                p.neighbors.append( Node(neighbor.val,[p]) )
                dfs( p.neighbors[-1] )
    dfs()
    return T
T = spanningTree(G)
for row in graph2Matrix(T,4):
    print(row)

[0, 1, 0, 1]
[1, 0, 1, 0]
[0, 1, 0, 0]
[1, 0, 0, 0]


In [163]:
# Minimum spanning tree: Leetcode medium 1/3k and matrix
"""
1. Kruskal algorithm: Easy to understand but hard to implement because of the weighted adjacent list + heap. O(elog(e))
    descending sort weights
    while edges<nodes:
        pop shortest weight
        if add the edge contains cycle:
            edges+=1
2. Prim algorithm: Easy to understand and implement. Matrix. O(v**2)
e.g. M                 min spannning tree
0 — (16) —  1          0 — (16) —  1
| \       / | \                  / | \
|  (21)(11) |  (5)            (11) |  (5)
|    \ /    |    \             /   |    \
(19)  5     (6)   2           5    (6)   2
|    /  \   |    /                 |   
|  (33)(14) |  (10)                | 
| /       \ | /                    |
4 — (18) —  3          4 — (18) —  3
"""
x = float('inf')
M = [
    [ x, 16,  x,  x, 19, 21],
    [16,  x,  5,  6,  x, 11],
    [ x,  5,  x, 10,  x,  x],
    [ x,  6, 10,  x, 18, 14],
    [19,  x,  x, 18,  x, 33],
    [21, 11,  x, 14, 33,  x],
]

"""Each vertex pick the smallest neighbor edge"""
def prim(M):
    T = [ [x]*len(M) for i in range(len(M)) ]
    for i in range(len(M)):
        rowMin = min(M[i])
        rowMinIdx = M[i].index( rowMin )
        T[i][rowMinIdx] = rowMin
        T[rowMinIdx][i] = rowMin
    return T
prim(M)

[[inf, 16, inf, inf, inf, inf],
 [16, inf, 5, 6, inf, 11],
 [inf, 5, inf, inf, inf, inf],
 [inf, 6, inf, inf, 18, inf],
 [inf, inf, inf, 18, inf, inf],
 [inf, 11, inf, inf, inf, inf]]

In [2]:
# Shortest path algorithm
# 1. Dijkstra: Greedy and non-negative edges only. O(v**2)
# Leetcode all are non-negative edges
x = float('inf')
M = [
    [ 0,  x,  x,  x,  x,  x,  x,  x],
    [ 3,  0,  x,  x,  x,  x,  x,  x],
    [10,  8,  0,  x,  x,  x,  x,  x],
    [ x,  x, 12,  0,  x,  x,  x,  x],
    [ x,  x,  x, 15,  0,  3,  x,  x],
    [ x,  x,  x, 10,  x,  0,  9, 14],
    [ x,  x,  x,  x,  x,  x,  0, 10],
    [17,  x,  x,  x,  x,  x,  x,  0],
]

def Dijkstra(M, start):
    dist  = M[start][:]
    visit = {start}
    for i in range(1,len(M)):
        # nextIndex: the unvisited smallest neighbor of i
        distTemp = [ float('inf') if j in visit else dist[j] for j in range(len(M)) ]
        nextIdx = distTemp.index( min(distTemp) )
        visit.add(nextIdx)
        for j in range(len(M)): # update all the neighbors of the nextIdx
            dist[j] = min(dist[j], dist[nextIdx]+M[nextIdx][j])
    return dist

print(Dijkstra(M,4)) # [34, 33, 25, 13, 0, 3, 12, 17]

[34, 33, 25, 13, 0, 3, 12, 17]


In [3]:
def DijkstraSpanning(M, start):
    dist  = M[start][:]
    visit = {start:[start]} # shortest path to each nodes
    for i in range(1,len(M)):
        # nextIndex: the unvisited smallest neighbor of i
        distTemp = [ float('inf') if j in visit else dist[j] for j in range(len(M)) ]
        nextIdx = distTemp.index( min(distTemp) )
        # parentIndex: minimum index of (current shortest path + each node to nextIdx) among all nodes except for itself  
        best2nextIdx = [ dist[j]+M[j][nextIdx] if j!=nextIdx else float('inf') for j in range(len(M)) ]
        parentIdx  = best2nextIdx.index( min(best2nextIdx) )
        visit[nextIdx] = visit[parentIdx]+[nextIdx]
        for j in range(len(M)): # update all the neighbors of the nextIdx
            dist[j] = min(dist[j], dist[nextIdx]+M[nextIdx][j])
    return dist, visit

dist, visit = DijkstraSpanning(M,4) 
print(dist) # [34, 33, 25, 13, 0, 3, 12, 17]
print(visit) # {4: [4], 5: [4, 5], 6: [4, 5, 6], 3: [4, 5, 3], 7: [4, 5, 7], 2: [4, 5, 3, 2], 1: [4, 5, 3, 2, 1], 0: [4, 5, 7, 0]}

[34, 33, 25, 13, 0, 3, 12, 17]
{4: [4], 5: [4, 5], 6: [4, 5, 6], 3: [4, 5, 3], 7: [4, 5, 7], 2: [4, 5, 3, 2], 1: [4, 5, 3, 2, 1], 0: [4, 5, 7, 0]}
